In [ ]:
import os
import polars as pl

In [ ]:
kaggle_run_type = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', None)
if kaggle_run_type:
    DATA_PATH = "/kaggle/input/linking-writing-processes-to-writing-quality"
else:
    DATA_PATH = "../../data"

In [ ]:
logs = pl.read_csv(f"{DATA_PATH}/train_logs.csv")

In [ ]:
logs.sample(3)

In [ ]:
def reconstruct_essay_text(single_log):
    essay_text = ""
    single_log = (
        single_log
        .filter(pl.col("activity").ne("Nonproduction"))
        .select("activity", "text_change", "cursor_position")
    )
    for row in single_log.iter_rows():
        activity = row[0]
        text_change = row[1]
        pos = row[2]
        
        if activity=="Input":
            input_text = text_change
            input_len = len(input_text)
            essay_text = f"{essay_text[:pos - input_len]}{input_text}{essay_text[pos - input_len:]}"
        elif activity=="Remove/Cut":
            removed_text = text_change
            removed_len = len(removed_text)
            essay_text = f"{essay_text[:pos]}{essay_text[pos + removed_len:]}"
        elif activity=="Replace":
            removed_text, input_text = text_change.split(" => ")
            removed_len = len(removed_text)
            input_len = len(input_text)
            essay_text = f"{essay_text[:pos - input_len]}{input_text}{essay_text[pos + removed_len - input_len:]}"
        elif activity=="Paste":
            input_text = text_change
            input_len = len(input_text)
            essay_text = f"{essay_text[:pos - input_len]}{input_text}{essay_text[pos - input_len:]}"
        elif activity[:10]=="Move From ":
            move_from, move_to = activity[10:].split(" To ")
            start_pos, end_pos = [int(x) for x in move_from[1:-1].split(", ")]
            target_pos, _ = [int(x) for x in move_to[1:-1].split(", ")]
            moved_text = text_change
            moved_len = len(text_change)
            essay_text = f"{essay_text[:start_pos]}{essay_text[end_pos:]}"
            essay_text = f"{essay_text[:target_pos]}{moved_text}{essay_text[target_pos:]}"
    return(essay_text)

Reconstructing all essays

In [ ]:
essays = {}

for essay_id in logs.select(pl.col("id").unique())["id"]:
    essay_text = reconstruct_essay_text(logs.filter(pl.col("id").eq(essay_id)))
    essays[essay_id] = essay_text

essays

The punctuation marks might be an indicator for the complexity of the essay.

In [ ]:
essays["e58d9b29"].replace("q", "").replace(" ", "")

In [ ]:
essays["5b3282ec"].replace("q", "").replace(" ", "")